# Water Distribution System Modeling and Analysis

This notebook walks through a complete water distribution system modeling workflow using the HydroFlow application. We'll cover data collection, processing, network modeling, hydraulic simulation, and visualization of results for a water distribution system in Madison, WI.

# Adding Project Root to the path

In [5]:
import os
import sys
# Get the absolute path to the project root
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
# Add the project root to Python's sys.path
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# Now you can import using the original paths
# This shows what's in your path
print(f"Project root added to path: {project_root}")
print(f"Current directory: {os.getcwd()}")

Project root added to path: /home/smdsgit/SouravDSGit/OpenHydroFlow
Current directory: /home/smdsgit/SouravDSGit/OpenHydroFlow/example


## 1. Setup and Environment Preparation

First, let's set up our environment, import necessary libraries, and configure logging.

In [11]:
# Import core libraries
import os
import logging
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import json
from pathlib import Path
import networkx as nx
from IPython.display import display, HTML

# Setup logging for better visibility in notebook
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# Get the project root directory
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Add project root to Python path for imports
import sys
if project_root not in sys.path:
    sys.path.insert(0, project_root)
    
print(f"Project root: {project_root}")

# Create necessary directory structure IN THE PROJECT ROOT
data_dirs = ["data/raw", "data/processed", "data/output", "epanet"]
for directory in data_dirs:
    # Use project_root to create directories in the right place
    full_path = os.path.join(project_root, directory)
    Path(full_path).mkdir(parents=True, exist_ok=True)
    print(f"Created directory: {full_path}")

print("Environment setup complete. Directory structure created.")

Project root: /home/smdsgit/SouravDSGit/OpenHydroFlow
Created directory: /home/smdsgit/SouravDSGit/OpenHydroFlow/data/raw
Created directory: /home/smdsgit/SouravDSGit/OpenHydroFlow/data/processed
Created directory: /home/smdsgit/SouravDSGit/OpenHydroFlow/data/output
Created directory: /home/smdsgit/SouravDSGit/OpenHydroFlow/epanet
Environment setup complete. Directory structure created.


## 2. Data Collection

The first step in water distribution modeling is gathering necessary data. We'll collect:
1. Water infrastructure GIS data (water mains, hydrants, pressure zones). 
    * The open data API documentation is available here : https://data-cityofmadison.opendata.arcgis.com/datasets/cityofmadison::water-main-breaks/about
    * For API go here: https://data-cityofmadison.opendata.arcgis.com/datasets/cityofmadison::water-main-breaks/api

2. USGS water data for streams and groundwater
3. EPA water quality data
4. Elevation data

Let's import our data collection module and collect the required data:

In [18]:
# Import the DataCollector class
from src.data_collection import DataCollector

# Create an instance of the DataCollector
collector = DataCollector()

# Fetch Madison water infrastructure GIS data
print("Collecting Madison water infrastructure GIS data...")
gis_data = collector.fetch_madison_water_gis()

# Display summary of GIS data collection results
if gis_data:
    print("\nGIS Data Collection Results:")
    for key, gdf in gis_data.items():
        if isinstance(gdf, gpd.GeoDataFrame):
            print(f"- {key}: {len(gdf)} features")
            # Show first few rows of each dataset
            display(gdf.head(2))

2025-04-01 11:32:33,238 - src.data_collection - INFO - EPA API key not provided. Using public access endpoints.
2025-04-01 11:32:33,239 - src.data_collection - INFO - USGS API key not provided. Using public access endpoints.
2025-04-01 11:32:33,240 - src.data_collection - INFO - Fetching Madison water infrastructure GIS data...
2025-04-01 11:32:33,240 - src.data_collection - INFO - Trying to download water mains from: https://data-cityofmadison.opendata.arcgis.com/datasets/cityofmadison::water-mains.geojson
2025-04-01 11:32:33,373 - src.data_collection - WARNING - Failed to download water mains from https://data-cityofmadison.opendata.arcgis.com/datasets/cityofmadison::water-mains.geojson: HTTP Error 500: Internal Server Error
2025-04-01 11:32:33,373 - src.data_collection - INFO - Trying to download water mains from: https://data-cityofmadison.opendata.arcgis.com/datasets/water-mains.geojson


2025-04-01 11:32:33,465 - src.data_collection - WARNING - Failed to download water mains from https://data-cityofmadison.opendata.arcgis.com/datasets/water-mains.geojson: HTTP Error 403: Forbidden
2025-04-01 11:32:33,467 - src.data_collection - INFO - Trying to download water mains from: https://data-cityofmadison.opendata.arcgis.com/api/download/v1/items/water-mains/geojson
2025-04-01 11:32:34,299 - src.data_collection - WARNING - Failed to download water mains from https://data-cityofmadison.opendata.arcgis.com/api/download/v1/items/water-mains/geojson: HTTP Error 400: Bad Request
2025-04-01 11:32:34,300 - src.data_collection - INFO - Trying to download hydrants from: https://data-cityofmadison.opendata.arcgis.com/datasets/cityofmadison::fire-hydrants.geojson
2025-04-01 11:32:34,404 - src.data_collection - WARNING - Failed to download hydrants from https://data-cityofmadison.opendata.arcgis.com/datasets/cityofmadison::fire-hydrants.geojson: HTTP Error 500: Internal Server Error
2025-


GIS Data Collection Results:
- water_mains: 10 features


,id,diameter_mm,length_m,roughness,geometry
0,M0,100,1000,100,"LINESTRING (-89.4512 43.0231, -89.4312 43.0431)"
1,M1,150,1100,100,"LINESTRING (-89.4262 43.0231, -89.4062 43.0431)"


- hydrants: 8 features


,id,status,geometry
0,H0,Active,POINT (-89.4412 43.0331)
1,H1,Active,POINT (-89.4162 43.0331)


- pressure_zones: 2 features


,id,name,pressure,geometry
0,Z0,Zone 1,40,"POLYGON ((-89.4612 43.0131, -89.4012 43.0131, ..."
1,Z1,Zone 2,50,"POLYGON ((-89.4312 43.0131, -89.3712 43.0131, ..."


### 2.1 USGS Water Data

Now, let's fetch USGS water data for the Madison area:

In [13]:
# Fetch USGS water data (streams and groundwater)
print("Collecting USGS water data...")
usgs_data = collector.fetch_usgs_water_data(days=30)

# Display summary of USGS data collection results
if usgs_data:
    print("\nUSGS Data Collection Results:")
    for key, df in usgs_data.items():
        if isinstance(df, pd.DataFrame):
            print(f"- {key}: {len(df)} records")
            # Show first few rows
            display(df.head(2))

2025-04-01 10:54:54,903 - src.data_collection - INFO - Fetching USGS water data for the past 30 days...
2025-04-01 10:54:54,904 - src.data_collection - INFO - Identifying USGS water monitoring sites in Madison area...


2025-04-01 10:54:55,615 - src.data_collection - WARNING - First method failed: Bad Request, check that your parameters are correct. URL: https://waterservices.usgs.gov/nwis/iv?format=json&bBox=-89.5417%2C43.0233%2C-89.2349%2C43.171&siteType=ST%2CGW&hasDataTypeCd=dv&siteStatus=active&startDT=site
2025-04-01 10:54:55,615 - src.data_collection - ERROR - Fallback method also failed: Query must specify a major filter: sites, stateCd, bBox, huc, or countyCd
2025-04-01 10:54:55,616 - src.data_collection - INFO - Creating sample USGS site data for Madison, WI...
2025-04-01 10:54:55,619 - src.data_collection - INFO - Created sample site data as fallback.
2025-04-01 10:54:55,622 - src.data_collection - INFO - Retrieving streamflow data for test sites: ['5430500', '5430501', '5430502']
2025-04-01 10:54:55,871 - src.data_collection - WARNING - Error getting streamflow data: Bad Request, check that your parameters are correct. URL: https://waterservices.usgs.gov/nwis/dv?format=json&parameterCd=0006


USGS Data Collection Results:
- streamflow: 155 records


,site_no,datetime,value,parameter_cd,qualifier_cd,agency_cd
0,5430500,2025-03-02 10:54:54.904721,51.828334,00060,,USGS
1,5430500,2025-03-03 10:54:54.904721,47.437599,00060,,USGS


### 2.2 EPA Water Quality Data

Next, we'll collect EPA water quality data:

In [19]:
# Fetch EPA water quality data
print("Collecting EPA water quality data...")
epa_data = collector.fetch_epa_water_quality()

# Display summary of EPA data collection
if isinstance(epa_data, pd.DataFrame):
    print(f"\nEPA Water Quality Data: {len(epa_data)} water systems")
    display(epa_data.head(2))

2025-04-01 11:33:13,008 - src.data_collection - INFO - Fetching EPA water quality data...
2025-04-01 11:33:13,009 - src.data_collection - INFO - Trying to fetch EPA data from: https://enviro.epa.gov/efservice/SDW_WATER_SYSTEM/PRIMACY_AGENCY_CODE/WI/CITY_NAME/MADISON/JSON


2025-04-01 11:33:13,512 - src.data_collection - WARNING - EPA API request failed (attempt 1/3): Expecting value: line 1 column 1 (char 0)
2025-04-01 11:33:14,513 - src.data_collection - INFO - Trying to fetch EPA data from: https://enviro.epa.gov/efservice/SDW_WATER_SYSTEM/PRIMACY_AGENCY_CODE/WI/CITY_NAME/MADISON/JSON
2025-04-01 11:33:14,816 - src.data_collection - WARNING - EPA API request failed (attempt 2/3): Expecting value: line 1 column 1 (char 0)
2025-04-01 11:33:16,818 - src.data_collection - INFO - Trying to fetch EPA data from: https://enviro.epa.gov/efservice/SDW_WATER_SYSTEM/PRIMACY_AGENCY_CODE/WI/CITY_NAME/MADISON/JSON
2025-04-01 11:33:16,994 - src.data_collection - WARNING - EPA API request failed (attempt 3/3): Expecting value: line 1 column 1 (char 0)
2025-04-01 11:33:16,996 - src.data_collection - ERROR - All attempts failed for https://enviro.epa.gov/efservice/SDW_WATER_SYSTEM/PRIMACY_AGENCY_CODE/WI/CITY_NAME/MADISON/JSON
2025-04-01 11:33:19,997 - src.data_collection 


EPA Water Quality Data: 3 water systems


,PWSID,PWS_NAME,CITY_NAME,STATE_CODE,SOURCE_WATER,POPULATION_SERVED_COUNT,PRIMARY_SOURCE_CODE,PRIMARY_SOURCE,EPA_REGION
0,WI5502001,MADISON WATER UTILITY 1,MADISON,WI,GW,250000,GW,Ground water,05
1,WI5502002,MADISON WATER UTILITY 2,MADISON,WI,GW,240000,GW,Ground water,05


### 2.3 Elevation Data

Finally, let's collect elevation data for the Madison area:

In [20]:
# Fetch elevation data
print("Collecting elevation data...")
elevation_path = collector.fetch_elevation_data()

if elevation_path:
    print(f"\nElevation data saved to: {elevation_path}")

2025-04-01 11:33:43,712 - src.data_collection - INFO - Fetching elevation data for Madison area...
2025-04-01 11:33:43,713 - src.data_collection - INFO - Trying dataset: Digital Elevation Model (DEM) 1/3 arc-second


2025-04-01 11:33:44,649 - src.data_collection - WARNING - No elevation data found for dataset: Digital Elevation Model (DEM) 1/3 arc-second
2025-04-01 11:33:44,650 - src.data_collection - INFO - Trying dataset: Digital Elevation Model (DEM) 1 arc-second
2025-04-01 11:33:45,367 - src.data_collection - WARNING - No elevation data found for dataset: Digital Elevation Model (DEM) 1 arc-second
2025-04-01 11:33:45,369 - src.data_collection - INFO - Trying dataset: Digital Elevation Model (DEM) 1 meter
2025-04-01 11:33:46,083 - src.data_collection - WARNING - No elevation data found for dataset: Digital Elevation Model (DEM) 1 meter
2025-04-01 11:33:46,085 - src.data_collection - WARNING - Could not download elevation data from any source. Creating sample elevation data.
2025-04-01 11:33:46,251 - src.data_collection - INFO - Created sample elevation data as fallback.



Elevation data saved to: data/raw/madison_elevation.tif


### 2.4 Complete Data Collection Summary

Let's fetch all data at once and summarize the results:

In [21]:
# Fetch all data at once
all_data = collector.fetch_all_data()

# Provide a summary of all collected data
print("Summary of all collected data:")
for category, data in all_data.items():
    print(f"\n{category.upper()}:")
    if category == 'gis':
        for layer_name, layer_data in data.items():
            if isinstance(layer_data, gpd.GeoDataFrame):
                print(f"  - {layer_name}: {len(layer_data)} features")
    elif category == 'usgs':
        for dataset, dataset_data in data.items():
            if isinstance(dataset_data, pd.DataFrame):
                print(f"  - {dataset}: {len(dataset_data)} records")
    elif category == 'epa':
        if isinstance(data, pd.DataFrame):
            print(f"  - Water systems: {len(data)} records")
    elif category == 'elevation':
        print(f"  - Elevation data: {data}")

2025-04-01 11:34:02,223 - src.data_collection - INFO - Starting complete data collection for Madison, WI...
2025-04-01 11:34:02,224 - src.data_collection - INFO - Fetching Madison water infrastructure GIS data...
2025-04-01 11:34:02,224 - src.data_collection - INFO - Trying to download water mains from: https://data-cityofmadison.opendata.arcgis.com/datasets/cityofmadison::water-mains.geojson
2025-04-01 11:34:02,366 - src.data_collection - WARNING - Failed to download water mains from https://data-cityofmadison.opendata.arcgis.com/datasets/cityofmadison::water-mains.geojson: HTTP Error 500: Internal Server Error
2025-04-01 11:34:02,367 - src.data_collection - INFO - Trying to download water mains from: https://data-cityofmadison.opendata.arcgis.com/datasets/water-mains.geojson
2025-04-01 11:34:02,462 - src.data_collection - WARNING - Failed to download water mains from https://data-cityofmadison.opendata.arcgis.com/datasets/water-mains.geojson: HTTP Error 403: Forbidden
2025-04-01 11:3

Summary of all collected data:

GIS:
  - water_mains: 10 features
  - hydrants: 8 features
  - pressure_zones: 2 features

USGS:
  - streamflow: 155 records

EPA:
  - Water systems: 3 records

ELEVATION:
  - Elevation data: data/raw/madison_elevation.tif


## 3. Data Processing

After collecting the raw data, we need to process it for use in our water distribution model. This includes:
1. Cleaning water mains data
2. Processing hydrants
3. Processing pressure zones
4. Extracting elevation data for junctions
5. Creating an EPANET network model

Let's import our data processing module and process the data:

In [22]:
# Import the DataProcessor class
from src.data_processing import DataProcessor

# Create an instance of the DataProcessor
processor = DataProcessor()

# Clean and prepare water mains data
print("Processing water mains data...")
water_mains = processor.clean_water_mains()

if isinstance(water_mains, gpd.GeoDataFrame):
    print(f"\nProcessed water mains: {len(water_mains)} pipes")
    display(water_mains.head(2))

2025-04-01 11:34:33,676 - src.data_processing - INFO - Cleaning water mains data...
2025-04-01 11:34:33,683 - src.data_processing - INFO - Original water mains data: 10 rows
2025-04-01 11:34:33,686 - src.data_processing - ERROR - Error processing water mains data: 'material'


Processing water mains data...


### 3.1 Processing Hydrants and Pressure Zones

In [ ]:
# Process hydrants data
print("Processing hydrants data...")
hydrants = processor.process_hydrants()

if isinstance(hydrants, gpd.GeoDataFrame):
    print(f"\nProcessed hydrants: {len(hydrants)} hydrants")
    display(hydrants.head(2))

# Process pressure zones data
print("\nProcessing pressure zones data...")
pressure_zones = processor.process_pressure_zones()

if isinstance(pressure_zones, gpd.GeoDataFrame):
    print(f"\nProcessed pressure zones: {len(pressure_zones)} zones")
    display(pressure_zones.head(2))

### 3.2 Creating a Network Model from Processed Data

Now we'll create a network model for EPANET from our processed data:

In [ ]:
# Create a subset area around downtown Madison for focused modeling
downtown_madison = (-89.3837, 43.0731)  # Longitude, Latitude
subset_area = processor.create_subset_area(downtown_madison, radius_km=3.0)

print("Creating EPANET network data...")
network_data = processor.create_epanet_network_data(water_mains, subset_area)

if network_data:
    print("\nEPANET Network Data:")
    print(f"- Junctions: {len(network_data['junctions'])} features")
    print(f"- Connections: {len(network_data['connections'])} features")
    if network_data['reservoir']:
        print(f"- Reservoir: {network_data['reservoir']['reservoir_id']} connected to {network_data['reservoir']['junction_id']}")
    
    # Display first few junctions
    display(network_data['junctions'].head(2))
    
    # Display first few connections
    display(pd.DataFrame(network_data['connections']).head(2))

### 3.3 Process All Data in One Step

Now let's run the full data processing pipeline:

In [ ]:
# Process all water distribution data with a spatial subset
print("Processing all water distribution data...")
processed_data = processor.process_all_data(subset_area)

if processed_data:
    print("\nComplete processed data summary:")
    for key, data in processed_data.items():
        if key == 'network_data':
            print(f"\n{key}:")
            print(f"  - Junctions: {len(data['junctions'])} features")
            print(f"  - Connections: {len(data['connections'])} features")
        elif isinstance(data, gpd.GeoDataFrame):
            print(f"\n{key}: {len(data)} features")

## 4. EPANET Setup

Before building our water distribution network model, we need to set up the EPANET command-line tool. EPANET is the hydraulic simulation engine we'll use to analyze water flow, pressure, and quality in our network.

In [ ]:
# Import the EPANET utility module
from src.epanet_util import setup_epanet, verify_executable

# Set up EPANET command-line tool
print("Setting up EPANET command-line tool...")
epanet_setup_success = setup_epanet()

if epanet_setup_success:
    print("\nEPANET setup successful.")
    
    # Verify EPANET executable
    if verify_executable():
        print("EPANET executable verification successful.")
    else:
        print("EPANET executable verification failed. A dummy executable was created.")
else:
    print("\nEPANET setup failed. Check logs for details.")

## 5. Network Modeling

Now we'll build a network model from our processed GIS data. This involves:
1. Creating a graph representation of the water distribution network
2. Adding elevation data
3. Adding junction demands
4. Adding water sources (reservoirs and tanks)
5. Generating an EPANET input file

In [ ]:
# Import the NetworkBuilder class
from src.network_model import NetworkBuilder

# Create an instance of the NetworkBuilder
builder = NetworkBuilder()

# Paths to processed data
mains_file = "data/processed/processed_water_mains.geojson"
hydrants_file = "data/processed/processed_hydrants.geojson"
pressure_zones_file = "data/processed/processed_pressure_zones.geojson"

# Build network model from GIS data
print("Building network model from GIS data...")
network_model = builder.build_from_gis(mains_file, hydrants_file, pressure_zones_file)

if network_model:
    print("\nNetwork model built successfully.")
    
    # Get network statistics
    stats = builder.get_network_stats(network_model)
    
    print("\nNetwork Statistics:")
    for key, value in stats.items():
        print(f"- {key}: {value}")
    
    # Save the network model
    model_file = "data/output/network_model.json"
    builder.save_network(network_model, model_file)
    print(f"\nNetwork model saved to {model_file}")

### 5.1 Visualizing the Network Graph Structure

In [ ]:
# Visualize network graph structure
if network_model:
    G = network_model['graph']
    
    # Get node positions for visualization
    pos = {}
    node_colors = []
    
    for node in G.nodes():
        # Get node attributes
        attrs = G.nodes[node]
        
        if 'x' in attrs and 'y' in attrs:
            pos[node] = (attrs['x'], attrs['y'])
        
        # Set node color based on type
        if attrs.get('type') == 'junction':
            node_colors.append('blue')
        elif attrs.get('type') == 'reservoir':
            node_colors.append('red')
        elif attrs.get('type') == 'tank':
            node_colors.append('green')
        else:
            node_colors.append('gray')
    
    # Create plot
    plt.figure(figsize=(12, 10))
    
    # Draw nodes
    nx.draw_networkx_nodes(G, pos, node_size=25, node_color=node_colors, alpha=0.7)
    
    # Draw edges
    nx.draw_networkx_edges(G, pos, width=0.5, alpha=0.5)
    
    # Add labels to reservoirs and tanks only
    labels = {node: node for node in G.nodes() 
              if G.nodes[node].get('type') in ['reservoir', 'tank']}
    nx.draw_networkx_labels(G, pos, labels=labels, font_size=10)
    
    plt.title("Water Distribution Network")
    plt.axis('off')
    plt.show()

## 6. Hydraulic Simulation

With our network model built, we can now run hydraulic simulations to analyze water flow, pressure, and other parameters throughout the network over time.

In [ ]:
# Import the EPANETSimulator class
from src.simulation import EPANETSimulator

# Create an instance of the EPANETSimulator
simulator = EPANETSimulator()

# Path to INP file
inp_file = "data/output/madison_network.inp"

# Run hydraulic simulation
print("Running hydraulic simulation...")
results = simulator.run_simulation(inp_file, duration_hours=24, report_time_step=1)

if results:
    print("\nSimulation completed successfully.")
    
    # Get simulation statistics
    stats = simulator.get_result_stats(results)
    
    print("\nSimulation Statistics:")
    print(f"- Duration: {stats['duration_hours']} hours")
    
    print("\nPressure (m):")
    print(f"- Min: {stats['pressure']['min']:.2f}")
    print(f"- Max: {stats['pressure']['max']:.2f}")
    print(f"- Avg: {stats['pressure']['avg']:.2f}")
    
    print("\nFlow (m³/s):")
    print(f"- Min: {stats['flow']['min']:.4f}")
    print(f"- Max: {stats['flow']['max']:.4f}")
    print(f"- Avg: {stats['flow']['avg']:.4f}")
    
    print("\nVelocity (m/s):")
    print(f"- Min: {stats['velocity']['min']:.2f}")
    print(f"- Max: {stats['velocity']['max']:.2f}")
    print(f"- Avg: {stats['velocity']['avg']:.2f}")
    
    # Save results
    results_file = "data/output/simulation_results.json"
    simulator.save_results(results, results_file)
    print(f"\nSimulation results saved to {results_file}")

### 6.1 Visualizing Time Series Simulation Results

In [ ]:
# Plot pressure and flow time series for selected nodes and pipes
if results:
    # Get time steps
    time_steps = results['time_steps']
    
    # Select a few junctions and pipes to plot
    junction_ids = list(results['nodes']['pressure'].keys())[:5]  # First 5 junctions
    pipe_ids = list(results['links']['flow'].keys())[:5]  # First 5 pipes
    
    # Create pressure plot
    plt.figure(figsize=(12, 6))
    
    for junction_id in junction_ids:
        pressures = results['nodes']['pressure'][junction_id]
        plt.plot(time_steps, pressures, marker='o', label=f"Junction {junction_id}")
    
    plt.title("Junction Pressures Over Time")
    plt.xlabel("Time (hours)")
    plt.ylabel("Pressure (m)")
    plt.legend()
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
    
    # Create flow plot
    plt.figure(figsize=(12, 6))
    
    for pipe_id in pipe_ids:
        flows = [abs(flow) for flow in results['links']['flow'][pipe_id]]  # Use absolute values
        plt.plot(time_steps, flows, marker='o', label=f"Pipe {pipe_id}")
    
    plt.title("Pipe Flow Rates Over Time")
    plt.xlabel("Time (hours)")
    plt.ylabel("Flow Rate (m³/s)")
    plt.legend()
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

## 7. Visualization

Finally, let's create visualizations for our water distribution model and simulation results:

In [ ]:
# Import the NetworkVisualizer class
from src.visualization import NetworkVisualizer

# Create an instance of the NetworkVisualizer
visualizer = NetworkVisualizer()

# Path to INP file and results file
inp_file = "data/output/madison_network.inp"
results_file = "data/output/simulation_results.json"

# Create GeoJSON representation of the network
print("Creating GeoJSON representation of the network...")
network_geojson = visualizer.get_network_geojson(inp_file)

if network_geojson:
    print("\nGeoJSON representation created successfully.")
    print(f"- Nodes: {len(network_geojson['nodes']['features'])} features")
    print(f"- Links: {len(network_geojson['links']['features'])} features")
    
    # Create network statistics charts
    charts = visualizer.create_network_stats_charts(inp_file)
    
    if charts:
        print("\nNetwork statistics charts created successfully.")
        
        # Display diameter distribution chart data
        print("\nPipe Diameter Distribution:")
        for i, label in enumerate(charts['diameter_distribution']['labels']):
            count = charts['diameter_distribution']['datasets'][0]['data'][i]
            print(f"- {label} mm: {count} pipes")

### 7.1 Creating Results Visualization

In [ ]:
# Create results visualization
print("Creating results visualization...")
visualization_data = visualizer.create_results_visualization(
    inp_file, 
    results_file, 
    output_file="data/output/visualization_data.json"
)

if visualization_data:
    print("\nResults visualization created successfully.")
    
    # Create results charts
    results_charts = visualizer.create_results_charts(results_file)
    
    if results_charts:
        print("\nResults charts created successfully.")

### 7.2 Visualizing the Network on a Map (Optional - Requires folium)

In [ ]:
# install folium if you want to visualize the network on a map

# try:
#     import folium
#     
#     # Create a map centered on Madison, WI
#     m = folium.Map(location=[43.0731, -89.4012], zoom_start=12)
#     
#     # Add nodes to the map
#     for feature in network_geojson['nodes']['features']:
#         coords = feature['geometry']['coordinates']
#         node_type = feature['properties'].get('type', 'unknown')
#         
#         # Set color based on node type
#         if node_type == 'junction':
#             color = 'blue'
#         elif node_type == 'reservoir':
#             color = 'red'
#         elif node_type == 'tank':
#             color = 'green'
#         else:
#             color = 'gray'
#         
#         folium.CircleMarker(
#             location=[coords[1], coords[0]],
#             radius=3,
#             color=color,
#             fill=True,
#             fill_opacity=0.7,
#             popup=feature['properties'].get('id', 'Unknown')
#         ).add_to(m)
#     
#     # Add pipes to the map
#     for feature in network_geojson['links']['features']:
#         coords = feature['geometry']['coordinates']
#         
#         # Convert coordinates format
#         line_coords = [[coord[1], coord[0]] for coord in coords]
#         
#         # Add line to map
#         folium.PolyLine(
#             line_coords,
#             color='black',
#             weight=2,
#             opacity=0.7,
#             popup=feature['properties'].get('id', 'Unknown')
#         ).add_to(m)
#     
#     # Display the map
#     display(m)
#     
# except ImportError:
#     print("Folium package not installed. Skip map visualization.")

## 8. Complete Pipeline and Summary

Let's run the complete pipeline from data collection to visualization:

In [ ]:
# Create directories
for directory in ["data/raw", "data/processed", "data/output", "epanet"]:
    Path(directory).mkdir(parents=True, exist_ok=True)

print("Starting complete water distribution modeling pipeline...")

# Step 1: Collect data
print("\n[Step 1] Collecting data...")
collector = DataCollector()
all_data = collector.fetch_all_data()

# Step 2: Process data
print("\n[Step 2] Processing data...")
processor = DataProcessor()
downtown_madison = (-89.3837, 43.0731)  # Longitude, Latitude
subset_area = processor.create_subset_area(downtown_madison, radius_km=3.0)
processed_data = processor.process_all_data(subset_area)

# Step 3: Set up EPANET
print("\n[Step 3] Setting up EPANET...")
setup_epanet()

# Step 4: Build network model
print("\n[Step 4] Building network model...")
builder = NetworkBuilder()
mains_file = "data/processed/processed_water_mains.geojson"
hydrants_file = "data/processed/processed_hydrants.geojson"
pressure_zones_file = "data/processed/processed_pressure_zones.geojson"
network_model = builder.build_from_gis(mains_file, hydrants_file, pressure_zones_file)

# Step 5: Run simulation
print("\n[Step 5] Running hydraulic simulation...")
simulator = EPANETSimulator()
inp_file = "data/output/madison_network.inp"
results = simulator.run_simulation(inp_file)

# Step 6: Create visualizations
print("\n[Step 6] Creating visualizations...")
visualizer = NetworkVisualizer()
results_file = "data/output/simulation_results.json"
visualization_data = visualizer.create_results_visualization(
    inp_file, 
    results_file, 
    output_file="data/output/visualization_data.json"
)

print("\nWater distribution modeling pipeline complete!")

## 9. Conclusion

In this notebook, we've completed the entire workflow for water distribution modeling:

1. **Data Collection**: Gathered water infrastructure GIS data, USGS water data, EPA water quality data, and elevation data.
2. **Data Processing**: Cleaned and prepared the data for modeling, including network topology creation.
3. **EPANET Setup**: Set up the EPANET hydraulic simulation engine.
4. **Network Modeling**: Built a complete water distribution network model from GIS data.
5. **Hydraulic Simulation**: Ran simulations to analyze water flow, pressure, and other parameters.
6. **Visualization**: Created visualizations of the network and simulation results.

The processed data and simulation results are available in the `data/processed` and `data/output` directories, respectively. These can be used for further analysis or integration with other systems.

### Next Steps and Potential Extensions

- **Water Quality Modeling**: Add water quality parameters to simulate chlorine decay or contaminant transport.
- **Scenario Analysis**: Explore different operational scenarios or emergency situations.
- **Real-time Integration**: Connect the model to SCADA or IoT sensor data for real-time monitoring.
- **Optimization**: Implement algorithms to optimize pump operations or water source blending.
- **Web Interface**: Develop a web-based dashboard for interactive exploration of the model and results.